#### The Two Wise Men Problem

Two wise men, Sage P and Sage S, are given two different natural numbers 𝑥 and 𝑦, where 1<𝑥,𝑦<100.

Sage P is told the product 𝑃=𝑥⋅𝑦, while Sage S is told the sum 𝑆=𝑥+𝑦. Neither of them knows the actual values of 𝑥 and 𝑦.

The following conversation takes place:

Sage P: "I don't know what the numbers are."

Sage S: "I knew that already."

Sage P: "Oh, now I know the numbers!"

Sage S: "And now I do too!"

What are the numbers 𝑥 and 𝑦?

In [12]:
limit = 100

# Створюємо словник для зберігання кількості входжень кожного добутку x * y.
product_counts = {}
for x in range(2, limit):
    for y in range(x + 1, limit - x):
        product = x * y
        product_counts[product] = product_counts.get(product, 0) + 1 

# Множина для заборонених сум (коли P не знає x та y)
not_allowed_sums = set()
for x in range(2, limit): 
    for y in range(x + 1, limit - x): 
        if product_counts[x * y] == 1:
            not_allowed_sums.add(x + y) 

# Підрахунок валідних добутків після виключення заборонених сум
valid_product_counts = {}
for n in range(2, limit):
    if n not in not_allowed_sums:
        for x in range(2, n // 2 + 1):
            product = x * (n - x)
            if product_counts.get(product, 0) > 1:
                valid_product_counts[product] = valid_product_counts.get(product, 0) + 1

# Визначаємо допустимі суми після другого висловлювання
valid_sums = set()
for n in range(2, limit):
    if n not in not_allowed_sums:
        unique_product = None
        for x in range(2, n // 2 + 1):
            product = x * (n - x)
            if valid_product_counts.get(product, 0) == 1:
                if unique_product is not None:
                    unique_product = None
                    break
                unique_product = product
        if unique_product is not None:
            valid_sums.add(n)

# Виводимо пари (сума, добуток) та відповідні значення x, y
for n in valid_sums:
    for x in range(2, n // 2 + 1):
        product = x * (n - x)
        if valid_product_counts.get(product, 0) == 1:
            print(f'(x, y) = ({x}, {n - x}) \n(Сума, Добуток) = ({n}, {product})')

(x, y) = (4, 13) 
(Сума, Добуток) = (17, 52)


In [13]:
import numpy as np

# Побудувати 2-й масив пар чисел на проміжку [2, 99]
# Зауваження: порядок чисел у парі не важливий, тобто пара (2, 3) є такою ж, як і пара (3, 2)
numbers = np.array([
    [a, b]
    for a in range(2, 100)
    for b in range(a, 100)
])

# Щоб зменшити діапазон можливих пар чисел, "Prod" може розглядати тільки цілі числа на проміжку [2, 99], добуток яких дорівнює заданому числу
# Кожен ключ у словнику відповідає 2D масиву пар чисел, добуток яких дорівнює цьому ключу
prods = {
    p: numbers[numbers.prod(axis=1) == p]
    for p in np.unique(numbers.prod(axis=1))
}

# "Sum" може робити те саме з заданим числом
# Кожен ключ у словнику відповідає 2D масиву пар чисел, сума яких дорівнює цьому ключу
sums = {
    s: numbers[numbers.sum(axis=1) == s]
    for s in np.unique(numbers.sum(axis=1))
}

# Виведення статистики
print("Усього є:")
print(f"- {len(numbers)} пар чисел на проміжку [2, 99]")
print(f"- {len(prods)} унікальних добутків цих пар чисел")
print(f"- {len(sums)} унікальних сум цих пар чисел")

Усього є:
- 4851 пар чисел на проміжку [2, 99]
- 2843 унікальних добутків цих пар чисел
- 195 унікальних сум цих пар чисел


In [14]:
# Оновлення пар чисел:
# Розглядаємо пари, пов'язані з кожною сумою в словнику `sums`, і зберігаємо лише ті пари, добутки яких мають більше ніж одну пару, що відповідає цьому добутку
numbers = np.concatenate([
    pairs  # Для кожної групи пар чисел
    for pairs in sums.values()  # Розглядаємо всі пари чисел з кожної суми
    if all(prods[p].shape[0] != 1 for p in pairs.prod(axis=1))  # Перевіряємо, що для кожного добутку є більше ніж одна пара
])

# Виведення кількості оновлених пар чисел
print(f"Кількість оновлених пар чисел: {len(numbers)}")

Кількість оновлених пар чисел: 145


In [15]:
# Визначаємо нові пари для добутків на основі оновлених пар чисел
prods = {
    p: numbers[numbers.prod(axis=1) == p]  # Для кожного унікального добутку створюємо відповідний масив пар
    for p in np.unique(numbers.prod(axis=1))  # Для кожного унікального добутку чисел
}

# Оновлюємо пари чисел:
# Розглядаємо пари, пов'язані з кожним добутком, і зберігаємо лише ті, які є єдиними для цього добутку
numbers = np.concatenate([
    pairs  # Для кожної групи пар чисел
    for pairs in prods.values()  # Розглядаємо всі пари чисел для кожного добутку
    if pairs.shape[0] == 1  # Залишаємо лише ті пари, які мають лише одну пару для цього добутку
])

# Виведення кількості оновлених пар чисел
print(f"Кількість оновлених пар чисел: {len(numbers)}")

Кількість оновлених пар чисел: 86


In [16]:
# Визначаємо нові пари для сум на основі оновлених пар чисел
sums = {
    s: numbers[numbers.sum(axis=1) == s]  # Для кожної унікальної суми створюємо відповідний масив пар
    for s in np.unique(numbers.sum(axis=1))  # Для кожної унікальної суми чисел
}

# Оновлюємо пари чисел:
# Розглядаємо пари, пов'язані з кожною сумою, і зберігаємо лише ті, які є єдиними для цієї суми
numbers = np.concatenate([
    pairs  # Для кожної групи пар чисел
    for pairs in sums.values()  # Розглядаємо всі пари чисел для кожної суми
    if pairs.shape[0] == 1  # Залишаємо лише ті пари, які мають лише одну пару для цієї суми
])

# Виведення кількості оновлених пар чисел
print(f"Кількість оновлених пар чисел: {len(numbers)}")

Кількість оновлених пар чисел: 1


In [17]:
# Виводимо числа, що залишились
a, b = np.squeeze(numbers)  # Розпаковуємо залишкові числа в окремі змінні a та b
print(f"Визначені числа: {a} та {b}")  # Виводимо визначені числа

Визначені числа: 4 та 13
